## Distributed ResNet Training with MXNet and Gluon

[ResNet_V2](https://arxiv.org/abs/1512.03385) is an architecture for deep convolution networks. In this example, we train a 34 layer network to perform image classification using the CIFAR-10 dataset. CIFAR-10 consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. 

In [1]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

# You must create a bucket that starts with 'deeplens' and has a unique name
bucket = 'deeplens-rd-east'

## Download training and test data

We use the helper scripts to download CIFAR10 training data and sample images.

In [2]:
from cifar10_utils import download_training_data
download_training_data()

downloading training data...
done


## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [3]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix='data/DEMO-gluon-cifar10')
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://deeplens-rd-east/data/DEMO-gluon-cifar10


## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The network itself is a pre-built version contained in the [Gluon Model Zoo](https://mxnet.incubator.apache.org/versions/master/api/python/gluon/model_zoo.html).

In [4]:
!cat 'cifar10.py'

from __future__ import print_function

import json
import logging
import os
import time

import mxnet as mx
from mxnet import autograd as ag
from mxnet import gluon
from mxnet.gluon.model_zoo import vision as models


# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #

def train(current_host, hosts, num_cpus, num_gpus, channel_input_dirs, model_dir, hyperparameters, **kwargs):
    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = hyperparameters.get('batch_size', 128)
    epochs = hyperparameters.get('epochs', 100)
    learning_rate = hyperparameters.get('learning_rate', 0.1)
    momentum = hyperparameters.get('momentum', 0.9)
    log_interval = hyperparameters.get('log_interval', 1)
    wd = hyperparameters.get('wd', 0.0001)

    if len(hosts) == 1:
        kvstore = 'device' if num_gpus > 0 else 'local'
 

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function as a distributed training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. In this case we will run our training job on two `ml.p2.xlarge` instances.

**Note:** you may need to request a limit increase in order to use two ``ml.p2.xlarge`` instances. If you 
want to try the example without requesting an increase, just change the ``train_instance_count`` value to ``1``.

In [5]:
m = MXNet("cifar10.py", 
          role=role, 
          base_job_name="deeplens",
          output_path='s3://' + bucket + "/model/",
          train_instance_count=1, 
          train_instance_type="ml.c5.4xlarge",
          framework_version="1.2.1",
          hyperparameters={'batch_size': 128, 
                           'epochs': 5, 
                           'learning_rate': 0.1, 
                           'momentum': 0.9})

No handlers could be found for logger "sagemaker"


After we've constructed our `MXNet` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.

In [6]:
m.fit(inputs)

2019-05-21 21:14:13 Starting - Starting the training job...
2019-05-21 21:14:14 Starting - Launching requested ML instances......
2019-05-21 21:15:29 Starting - Preparing the instances for training.........
2019-05-21 21:16:46 Downloading - Downloading input data...
2019-05-21 21:17:26 Training - Downloading the training image...
2019-05-21 21:17:59 Training - Training image download completed. Training in progress.
2019-05-21 21:17:59,435 INFO - root - running container entrypoint
2019-05-21 21:17:59,436 INFO - root - starting train task
2019-05-21 21:17:59,457 INFO - container_support.training - Training starting
2019-05-21 21:18:00,593 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/

2019-05-21 21:18:19,619 INFO - root - Epoch [0] Batch [127]#011Speed: 1451.956047 samples/sec#011accuracy=0.197571
2019-05-21 21:18:19,705 INFO - root - Epoch [0] Batch [128]#011Speed: 1481.309251 samples/sec#011accuracy=0.198098
2019-05-21 21:18:19,791 INFO - root - Epoch [0] Batch [129]#011Speed: 1491.146548 samples/sec#011accuracy=0.198738
2019-05-21 21:18:19,878 INFO - root - Epoch [0] Batch [130]#011Speed: 1485.403610 samples/sec#011accuracy=0.199129
2019-05-21 21:18:19,964 INFO - root - Epoch [0] Batch [131]#011Speed: 1484.783291 samples/sec#011accuracy=0.200225
2019-05-21 21:18:20,052 INFO - root - Epoch [0] Batch [132]#011Speed: 1461.489641 samples/sec#011accuracy=0.200129
2019-05-21 21:18:20,138 INFO - root - Epoch [0] Batch [133]#011Speed: 1479.154370 samples/sec#011accuracy=0.201201
2019-05-21 21:18:20,228 INFO - root - Epoch [0] Batch [134]#011Speed: 1435.779324 samples/sec#011accuracy=0.202141
2019-05-21 21:18:20,314 INFO - root - Epoch [0] Batch [135]#011Speed: 1481.87349

2019-05-21 21:18:29,581 INFO - root - Epoch [0] Batch [241]#011Speed: 1474.822847 samples/sec#011accuracy=0.249128
2019-05-21 21:18:29,669 INFO - root - Epoch [0] Batch [242]#011Speed: 1465.151072 samples/sec#011accuracy=0.249357
2019-05-21 21:18:29,757 INFO - root - Epoch [0] Batch [243]#011Speed: 1458.190952 samples/sec#011accuracy=0.249712
2019-05-21 21:18:29,845 INFO - root - Epoch [0] Batch [244]#011Speed: 1458.908170 samples/sec#011accuracy=0.250159
2019-05-21 21:18:29,933 INFO - root - Epoch [0] Batch [245]#011Speed: 1455.191055 samples/sec#011accuracy=0.250603
2019-05-21 21:18:30,020 INFO - root - Epoch [0] Batch [246]#011Speed: 1467.991491 samples/sec#011accuracy=0.250886
2019-05-21 21:18:30,108 INFO - root - Epoch [0] Batch [247]#011Speed: 1469.373606 samples/sec#011accuracy=0.251544
2019-05-21 21:18:30,195 INFO - root - Epoch [0] Batch [248]#011Speed: 1467.770754 samples/sec#011accuracy=0.251883
2019-05-21 21:18:30,286 INFO - root - Epoch [0] Batch [249]#011Speed: 1415.64575

2019-05-21 21:18:39,620 INFO - root - Epoch [0] Batch [355]#011Speed: 1456.679578 samples/sec#011accuracy=0.290511
2019-05-21 21:18:39,708 INFO - root - Epoch [0] Batch [356]#011Speed: 1469.707472 samples/sec#011accuracy=0.290988
2019-05-21 21:18:39,795 INFO - root - Epoch [0] Batch [357]#011Speed: 1474.993027 samples/sec#011accuracy=0.291354
2019-05-21 21:18:39,882 INFO - root - Epoch [0] Batch [358]#011Speed: 1463.944526 samples/sec#011accuracy=0.291522
2019-05-21 21:18:39,970 INFO - root - Epoch [0] Batch [359]#011Speed: 1464.495615 samples/sec#011accuracy=0.291927
2019-05-21 21:18:40,058 INFO - root - Epoch [0] Batch [360]#011Speed: 1456.446425 samples/sec#011accuracy=0.292265
2019-05-21 21:18:40,146 INFO - root - Epoch [0] Batch [361]#011Speed: 1465.791109 samples/sec#011accuracy=0.292645
2019-05-21 21:18:40,233 INFO - root - Epoch [0] Batch [362]#011Speed: 1467.654392 samples/sec#011accuracy=0.292872
2019-05-21 21:18:40,321 INFO - root - Epoch [0] Batch [363]#011Speed: 1463.74495

2019-05-21 21:18:49,647 INFO - root - Epoch [1] Batch [50]#011Speed: 1475.365940 samples/sec#011accuracy=0.430147
2019-05-21 21:18:49,733 INFO - root - Epoch [1] Batch [51]#011Speed: 1482.065762 samples/sec#011accuracy=0.430138
2019-05-21 21:18:49,821 INFO - root - Epoch [1] Batch [52]#011Speed: 1466.728169 samples/sec#011accuracy=0.430277
2019-05-21 21:18:49,909 INFO - root - Epoch [1] Batch [53]#011Speed: 1462.588538 samples/sec#011accuracy=0.430411
2019-05-21 21:18:49,995 INFO - root - Epoch [1] Batch [54]#011Speed: 1483.023753 samples/sec#011accuracy=0.430682
2019-05-21 21:18:50,082 INFO - root - Epoch [1] Batch [55]#011Speed: 1482.323561 samples/sec#011accuracy=0.430804
2019-05-21 21:18:50,169 INFO - root - Epoch [1] Batch [56]#011Speed: 1471.855071 samples/sec#011accuracy=0.432292
2019-05-21 21:18:50,256 INFO - root - Epoch [1] Batch [57]#011Speed: 1481.068148 samples/sec#011accuracy=0.430361
2019-05-21 21:18:50,343 INFO - root - Epoch [1] Batch [58]#011Speed: 1473.701450 samples

...

2019-05-21 21:20:24,703 INFO - root - Epoch [3] Batch [289]#011Speed: 1461.720431 samples/sec#011accuracy=0.581196
2019-05-21 21:20:24,793 INFO - root - Epoch [3] Batch [290]#011Speed: 1437.959781 samples/sec#011accuracy=0.581266
2019-05-21 21:20:24,881 INFO - root - Epoch [3] Batch [291]#011Speed: 1462.174267 samples/sec#011accuracy=0.581336
2019-05-21 21:20:24,968 INFO - root - Epoch [3] Batch [292]#011Speed: 1472.194102 samples/sec#011accuracy=0.581485
2019-05-21 21:20:25,055 INFO - root - Epoch [3] Batch [293]#011Speed: 1473.992768 samples/sec#011accuracy=0.581234
2019-05-21 21:20:25,143 INFO - root - Epoch [3] Batch [294]#011Speed: 1457.692090 samples/sec#011accuracy=0.581276
2019-05-21 21:20:25,230 INFO - root - Epoch [3] Batch [295]#011Speed: 1467.606248 samples/sec#011accuracy=0.581398
2019-05-21 21:20:25,319 INFO - root - Epoch [3] Batch [296]#011Speed: 1453.536332 samples/sec#011accuracy=0.581413
2019-05-21 21:20:25,407 INFO - root - Epoch [3] Batch [297]#011Speed: 1447.86412

2019-05-21 21:20:36,042 INFO - root - [Epoch 3] validation: accuracy=0.573518
2019-05-21 21:20:36,165 INFO - root - Epoch [4] Batch [1]#011Speed: 1473.891603 samples/sec#011accuracy=0.652344
2019-05-21 21:20:36,253 INFO - root - Epoch [4] Batch [2]#011Speed: 1463.090758 samples/sec#011accuracy=0.648438
2019-05-21 21:20:36,344 INFO - root - Epoch [4] Batch [3]#011Speed: 1419.793861 samples/sec#011accuracy=0.640625
2019-05-21 21:20:36,432 INFO - root - Epoch [4] Batch [4]#011Speed: 1460.638733 samples/sec#011accuracy=0.637500
2019-05-21 21:20:36,520 INFO - root - Epoch [4] Batch [5]#011Speed: 1456.513597 samples/sec#011accuracy=0.630208
2019-05-21 21:20:36,607 INFO - root - Epoch [4] Batch [6]#011Speed: 1466.259489 samples/sec#011accuracy=0.631696
2019-05-21 21:20:36,697 INFO - root - Epoch [4] Batch [7]#011Speed: 1427.138397 samples/sec#011accuracy=0.625977
2019-05-21 21:20:36,785 INFO - root - Epoch [4] Batch [8]#011Speed: 1457.411135 samples/sec#011accuracy=0.625868
2019-05-21 21:20:3

2019-05-21 21:20:50,718 INFO - root - Epoch [4] Batch [166]#011Speed: 1406.282676 samples/sec#011accuracy=0.623690
2019-05-21 21:20:50,808 INFO - root - Epoch [4] Batch [167]#011Speed: 1421.007681 samples/sec#011accuracy=0.623651
2019-05-21 21:20:50,896 INFO - root - Epoch [4] Batch [168]#011Speed: 1455.636898 samples/sec#011accuracy=0.623891
2019-05-21 21:20:50,984 INFO - root - Epoch [4] Batch [169]#011Speed: 1464.579513 samples/sec#011accuracy=0.623713
2019-05-21 21:20:51,072 INFO - root - Epoch [4] Batch [170]#011Speed: 1454.016602 samples/sec#011accuracy=0.623766
2019-05-21 21:20:51,161 INFO - root - Epoch [4] Batch [171]#011Speed: 1445.412235 samples/sec#011accuracy=0.623728
2019-05-21 21:20:51,249 INFO - root - Epoch [4] Batch [172]#011Speed: 1463.110694 samples/sec#011accuracy=0.624142
2019-05-21 21:20:51,336 INFO - root - Epoch [4] Batch [173]#011Speed: 1463.026965 samples/sec#011accuracy=0.623878
2019-05-21 21:20:51,424 INFO - root - Epoch [4] Batch [174]#011Speed: 1462.77184


2019-05-21 21:21:17 Uploading - Uploading generated training model2019-05-21 21:21:00,690 INFO - root - Epoch [4] Batch [278]#011Speed: 1458.460321 samples/sec#011accuracy=0.627296
2019-05-21 21:21:00,778 INFO - root - Epoch [4] Batch [279]#011Speed: 1461.907505 samples/sec#011accuracy=0.627232
2019-05-21 21:21:00,866 INFO - root - Epoch [4] Batch [280]#011Speed: 1454.710010 samples/sec#011accuracy=0.627419
2019-05-21 21:21:00,954 INFO - root - Epoch [4] Batch [281]#011Speed: 1457.628766 samples/sec#011accuracy=0.627327
2019-05-21 21:21:01,043 INFO - root - Epoch [4] Batch [282]#011Speed: 1449.588137 samples/sec#011accuracy=0.627264
2019-05-21 21:21:01,131 INFO - root - Epoch [4] Batch [283]#011Speed: 1452.250615 samples/sec#011accuracy=0.627448
2019-05-21 21:21:01,223 INFO - root - Epoch [4] Batch [284]#011Speed: 1398.662255 samples/sec#011accuracy=0.627632
2019-05-21 21:21:01,312 INFO - root - Epoch [4] Batch [285]#011Speed: 1436.959102 samples/sec#011accuracy=0.627622
2019-05-21 21

2019-05-21 21:21:13,194 INFO - root - [Epoch 4] validation: accuracy=0.622796

2019-05-21 21:21:37 Completed - Training job completed
Billable seconds: 292
